In [16]:
import json
import pandas as pd
import calendar

In [2]:
with open("../data/raw/banana.json", 'r') as j:
     contents = json.loads(j.read())

df = pd.DataFrame(contents["results"])
df

,annee,mois,regroupement1,regroupement2,poids_enquete_kg,poids_mg_kg,valeur_tot_mg_fr,valeur_moy
0,2012,1,BANANE DESSERT,TOTAL FRUITS (HORS VANILLE),108900.0,45602.00000,11327472.0,248.398579
1,2012,2,BANANE DESSERT,TOTAL FRUITS (HORS VANILLE),141065.0,38345.80078,8873594.0,231.409798
2,2012,3,BANANE DESSERT,TOTAL FRUITS (HORS VANILLE),134653.0,49828.83984,11063095.0,222.021926
3,2012,4,BANANE DESSERT,TOTAL FRUITS (HORS VANILLE),98451.0,30571.00000,7275857.0,237.998659
4,2012,5,BANANE DESSERT,TOTAL FRUITS (HORS VANILLE),76918.0,24259.00000,6223918.0,256.561194
...,...,...,...,...,...,...,...,...
134,2023,3,BANANE DESSERT,TOTAL FRUITS (HORS VANILLE),90131.0,33072.20000,10711771.0,323.890488
135,2023,4,BANANE DESSERT,TOTAL FRUITS (HORS VANILLE),75028.0,24272.80000,8039015.0,331.194382
136,2023,5,BANANE DESSERT,TOTAL FRUITS (HORS VANILLE),73147.0,27459.50000,9319075.0,339.375262
137,2023,6,BANANE DESSERT,TOTAL FRUITS (HORS VANILLE),70847.0,18675.80000,6184225.0,331.135748


In [17]:
# Cast columns
list_col_to_cast = ["annee", "mois"]
for col in list_col_to_cast:
    df[col] = df[col].astype(int)
df["mois_nom"] = df["mois"].map(lambda x: calendar.month_name[x])

In [4]:
# Check valeur_moy column
df["calculated_value"] = df["valeur_tot_mg_fr"] / df["poids_mg_kg"]
df["pb_value"] = df["valeur_moy"] - df["calculated_value"]
df[df["pb_value"]!=0].shape[0]

0

In [5]:
# Explode dataset
df_pivoted = df.pivot_table(values="poids_enquete_kg", index="annee", columns="mois")
df_pivoted

mois,1,2,3,4,5,6,7,8,9,10,11,12
annee,,,,,,,,,,,,
2012,108900.0,141065.0,134653.0,98451.0,76918.0,71272.0,59162.6,67010.0,60706.0,51054.0,64469.0,91305.5
2013,112403.0,97100.0,92166.0,66802.0,79905.0,78580.0,80185.0,74950.0,73830.0,78426.0,91575.0,118397.0
2014,125552.0,127276.0,93831.0,79034.0,71608.0,77628.0,63243.0,55611.0,60596.0,63713.0,81705.0,92432.0
2015,105376.0,118668.0,132359.0,102292.0,77251.0,52487.0,47387.0,54259.0,57112.0,84786.0,83381.0,100165.0
2016,117783.0,138598.0,134617.0,122086.3,112438.0,64650.0,70282.0,74473.0,77653.0,77750.0,81081.0,97892.0
2017,94995.0,116290.0,120558.0,68568.0,29635.0,21681.0,17755.0,16863.0,21482.0,23047.0,29508.0,56898.0
2018,52933.0,61339.0,65979.0,64437.0,63913.0,56992.2,63437.0,68567.0,77855.0,75028.0,83726.0,107113.0
2019,118592.0,106119.0,78995.0,100141.0,93741.0,70633.0,65875.0,57747.0,60795.0,83921.0,98310.0,99155.0
2020,118458.0,133922.0,126078.0,99176.0,75285.0,81885.0,65965.0,67604.0,90471.0,111485.0,133643.0,133755.0


In [18]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'polar'}]*1]*1)
df_2022 = df[df["annee"]==2022]
fig.add_trace(go.Scatterpolar(
      name = "angular categories",
      r = df_2022["poids_enquete_kg"],
      theta = df_2022["mois_nom"].astype("str"),
    ), 1, 1)
fig.show()

In [37]:
# Graph
import plotly.express as px

fig = px.line_polar(df, r="poids_enquete_kg", theta="mois_nom", color="annee",
                    color_discrete_sequence=px.colors.sequential.Plotly3, line_close=True,
                    template="plotly_dark",line_shape='spline')
fig.show()

In [36]:
fig = px.line_polar(df, r="valeur_moy", theta="mois_nom", color="annee",
                    color_discrete_sequence=px.colors.sequential.Plotly3, line_close=True,
                    template="plotly_dark",line_shape='spline')
fig.show()

# Get Data

In [77]:
from kedro.extras.datasets.api import APIDataSet


def get_data_with_filter(list_items: list, filter_name: str, limit: int= 100, offset: int=0):
    data_set = APIDataSet(
        url="https://data.gouv.nc/api/explore/v2.1/catalog/datasets/pacificdatavizchallenge_fruits_et_legumes_regroupement1/records",
        params={
            "select": "annee, mois, regroupement1, regroupement2, poids_enquete_kg, poids_mg_kg, valeur_tot_mg_fr, valeur_moy",
            "order_by": "annee, mois",
            "where": f'regroupement1 like "{filter_name}"',
            "limit": limit,
            "offset": offset
        }
    )
    raw_result = data_set.load().json()
    nb_items = raw_result["total_count"]
    list_items += raw_result["results"]
    return nb_items, list_items

def get_all_items(filter_name: str):
    nb_items_collected = 0
    nb_items_to_collect = 0
    chunk_size = 100
    list_items = []
    while nb_items_collected <= nb_items_to_collect:
        nb_items_to_collect, list_items = get_data_with_filter(list_items, filter_name, limit=chunk_size, offset=nb_items_collected)
        nb_items_collected += chunk_size
    return list_items

def transform_listdict_to_df(list_dict):
    return pd.DataFrame(list_dict)


In [79]:
list_objs = get_all_items(filter_name="CONCOMBRE")
df = transform_listdict_to_df(list_objs)
df.shape

(139, 8)

In [80]:
df

,annee,mois,regroupement1,regroupement2,poids_enquete_kg,poids_mg_kg,valeur_tot_mg_fr,valeur_moy
0,2012,1,CONCOMBRE,TOTAL LEGUMES,48497.0,71719.0000,11675743.0,162.798463
1,2012,2,CONCOMBRE,TOTAL LEGUMES,60714.0,38774.0000,6712380.0,173.115490
2,2012,3,CONCOMBRE,TOTAL LEGUMES,101864.0,83313.0000,12227620.0,146.767251
3,2012,4,CONCOMBRE,TOTAL LEGUMES,68808.0,72538.5293,8604470.5,118.619313
4,2012,5,CONCOMBRE,TOTAL LEGUMES,85703.0,61867.5000,8537520.0,137.996848
...,...,...,...,...,...,...,...,...
134,2023,3,CONCOMBRE,TOTAL LEGUMES,97518.0,34331.6000,8691833.0,253.172966
135,2023,4,CONCOMBRE,TOTAL LEGUMES,100784.9,26062.8000,7275865.0,279.166667
136,2023,5,CONCOMBRE,TOTAL LEGUMES,57879.0,13642.7000,3978896.0,291.650187
137,2023,6,CONCOMBRE,TOTAL LEGUMES,66005.0,12271.9000,5983896.0,487.609580
